In [1]:
import chemsource as cs
import pandas as pd
import re
from tqdm import tqdm
from ast import literal_eval
from tqdm import tqdm

from helpers import *

tqdm.pandas()

Configure chemsource Model

In [2]:
user_openai_key = pd.read_csv("api_keys.txt", header=None).values[0][0]
user_ncbi_key = pd.read_csv("api_keys.txt", header=None).values[1][0]

model = cs.ChemSource()
model.configure(openai_key=user_openai_key, ncbi_key=user_ncbi_key)

model.configuration()

{'openai_key': '********************************************************************************************************************************************************************',
 'ncbi_key': '************************************',
 'model': 'gpt-4-0125-preview',
 'prompt': 'Classify this compound, COMPOUND_NAME, as any combination of the following: MEDICAL, ENDOGENOUS, FOOD, PERSONAL CARE, INDUSTRIAL. Note that ENDOGENOUS refers to compounds that are human synthesized. ENDOGENOUS excludes essential nutrients that cannot be synthesized by human body. Note that FOOD refers to compounds present in natural food items. Note that INDUSTRIAL should be used only for compounds not used as a contributing ingredient in the medical, personal care, or food industries. Note that PERSONAL CARE refers to non-medicated compounds typically used for activities such as skincare, beauty, and fitness. Specify INFO instead if more information is needed. DO NOT MAKE ANY ASSUMPTIONS, USE ONLY THE INFORMAT

Read and Preprocess Data

In [6]:
raw_rosmap_data = pd.read_csv('data_in/ROSMAP_annotation_full_metadata_cleaned.tsv', sep='\t')
rosmap_data = raw_rosmap_data.copy()
rosmap_data["synonyms"] = raw_rosmap_data["synonyms"].apply(literal_eval)   

print(f"Length of unfiltered dataset : {len(rosmap_data)}")
rosmap_data.drop_duplicates(subset=["compound_name"], inplace=True)
print(f"Length of dataset (no duplicates): {len(rosmap_data)}")

rosmap_data["synonyms"] = rosmap_data["synonyms"].apply(lambda x: filter_synonym_list(x))
rosmap_data["synonyms"] = rosmap_data["synonyms"].apply(lambda x: preprocessing_function_synonyms(x))
rosmap_data = rosmap_data[rosmap_data["synonyms"].map(len) > 0]

rosmap_data["synonyms"] = rosmap_data["synonyms"].apply(lambda x: x[:6] if isinstance(x, list) else x)

print(f"Length of dataset (after filtering): {len(rosmap_data)}")

Length of unfiltered dataset : 229
Length of dataset (no duplicates): 174
Length of dataset (after filtering): 153


Run Model and Output Data

In [14]:
#Code is commented out to avoid running multiple times and wasting API credits
# rosmap_data["chemsource_raw_output"] = rosmap_data["synonyms"].progress_apply(lambda x : chemsource_list_apply(x, model))
# rosmap_data.to_csv("./data_out/rosmap_data_chemsource_raw_output.tsv", sep="\t")

 25%|██▌       | 39/153 [01:34<05:30,  2.90s/it]/home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/prajitrr/miniconda3/envs/chemsource/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 153/153 [06:57<00:00,  2.73s/it]


Process Raw Output

In [34]:
rosmap_data_raw_output = pd.read_csv("./data_out/rosmap_data_chemsource_raw_output.tsv", sep="\t", index_col=0)
rosmap_data_raw_output["chemsource_raw_output"] = rosmap_data_raw_output["chemsource_raw_output"].apply(literal_eval)
rosmap_data_raw_output["synonyms"] = rosmap_data_raw_output["synonyms"].apply(literal_eval)   

rosmap_data_raw_output["name_used"] = rosmap_data_raw_output["chemsource_raw_output"].apply(lambda x: x[0])
rosmap_data_raw_output["source"] = rosmap_data_raw_output["chemsource_raw_output"].apply(lambda x: x[1][0][0])
rosmap_data_raw_output["text"] = rosmap_data_raw_output["chemsource_raw_output"].apply(lambda x: x[1][0][1])
rosmap_data_raw_output["text_length"] = rosmap_data_raw_output["text"].apply(lambda x: get_length(x))
rosmap_data_raw_output["classification"] = rosmap_data_raw_output["chemsource_raw_output"].apply(lambda x: x[1][1])

In [ ]:
#Code is commented out to avoid running multiple times
# rosmap_data_raw_output.to_csv("./data_out/rosmap_data_final_output.tsv", sep="\t")